In [1]:
import tensorflow as tf
import input_data

In [2]:
mnist = input_data.read_data_sets("tf_mnist", one_hot='True')

Extracting tf_mnist\train-images-idx3-ubyte.gz
Extracting tf_mnist\train-labels-idx1-ubyte.gz
Extracting tf_mnist\t10k-images-idx3-ubyte.gz
Extracting tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
learing_rate = 1e-3
n_train_size = 200000
batch_size = 128
display_step = 50

In [4]:
n_image_size = 28
n_input = n_image_size * n_image_size
n_classes = 10
dropout = 0.75
n_fc1_size = int(n_input/16)

In [5]:
def conv2d(x, W, b, strides=1):
    output = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    output = tf.nn.relu( output + b )
    return output    

In [6]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], 
               padding='SAME')

In [7]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, n_image_size, n_image_size, 1])
    
    h1 = maxpool2d( conv2d(x,  weights['h1'], biases['h1']) )    
    h2 = maxpool2d( conv2d(h1, weights['h2'], biases['h2']) )
    
    fc1 = tf.reshape(h2, [-1, weights['fc1'].get_shape().as_list()[0]])
    fc1 = tf.nn.relu( tf.matmul(fc1, weights['fc1']) + biases['fc1'] )
    fc1 = tf.nn.dropout(fc1, dropout)
    
    return tf.matmul(fc1, weights['out']) + biases['out']        

In [8]:
graph = tf.Graph()
with graph.as_default():
    
    Xtr = tf.placeholder(tf.float32, [None, n_input])
    Ytr = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    weights = {
        "h1": tf.Variable(tf.random_normal([5, 5, 1, 32])),
        "h2": tf.Variable(tf.random_normal([5, 5, 32, 64])),
        "fc1": tf.Variable(tf.random_normal([n_fc1_size*64, 1024])),
        "out": tf.Variable(tf.random_normal([1024, n_classes]))
    }
    
    biases = {
        "h1": tf.Variable(tf.zeros([32])),
        "h2": tf.Variable(tf.zeros([64])),
        "fc1": tf.Variable(tf.zeros([1024])),
        "out": tf.Variable(tf.zeros([n_classes]))
    }

    Ypred = conv_net(Xtr, weights, biases, keep_prob)
    
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(
        logits=Ypred, labels=Ytr ))
    tf_train = tf.train.AdamOptimizer(learing_rate).minimize(cost)
    
    correct_pred = tf.equal( tf.argmax(Ypred, 1), tf.argmax(Ytr, 1) )
    train_acc = tf.reduce_mean( tf.cast(correct_pred, tf.float32) )

In [9]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    train_step = 1
    while train_step * batch_size < n_train_size:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(tf_train, feed_dict={
            Xtr: batch_x, Ytr: batch_y, keep_prob: dropout })
        if train_step % display_step == 0:
            cost_, train_acc_ = sess.run([cost, train_acc], feed_dict={
                Xtr: batch_x, Ytr: batch_y, keep_prob: 1.0 })
            print ("Iter: %d, loss: %.4f, train accuracy: %.4f"% 
                  (train_step*batch_size, cost_, train_acc_))
        train_step += 1
    
    print ("Optimization Finished!")
    train_acc_ = train_acc.eval(feed_dict={
        Xtr: mnist.test.images[:batch_size*2],
        Ytr: mnist.test.labels[:batch_size*2],
        keep_prob: 1.0 })
    print ("Test Accuracy: %.4f"% (train_acc_))            

Iter: 6400, loss: 3009.7063, train accuracy: 0.8516
Iter: 12800, loss: 3012.7568, train accuracy: 0.8281
Iter: 19200, loss: 895.7406, train accuracy: 0.8906
Iter: 25600, loss: 964.8881, train accuracy: 0.9141
Iter: 32000, loss: 789.8167, train accuracy: 0.9297
Iter: 38400, loss: 418.6167, train accuracy: 0.9609
Iter: 44800, loss: 212.3507, train accuracy: 0.9766
Iter: 51200, loss: 532.3846, train accuracy: 0.9531
Iter: 57600, loss: 903.1747, train accuracy: 0.9062
Iter: 64000, loss: 545.1428, train accuracy: 0.9609
Iter: 70400, loss: 747.3668, train accuracy: 0.9609
Iter: 76800, loss: 442.9000, train accuracy: 0.9453
Iter: 83200, loss: 491.5665, train accuracy: 0.9375
Iter: 89600, loss: 473.8859, train accuracy: 0.9609
Iter: 96000, loss: 1103.5457, train accuracy: 0.9375
Iter: 102400, loss: 72.3549, train accuracy: 0.9766
Iter: 108800, loss: 222.1935, train accuracy: 0.9766
Iter: 115200, loss: 206.4553, train accuracy: 0.9766
Iter: 121600, loss: 55.9769, train accuracy: 0.9922
Iter: 12